In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense, Dropout, BatchNormalization
from keras.preprocessing.sequence import TimeseriesGenerator

%matplotlib inline  




In [32]:
#yf.download('GE',start='2019-06-01', interval='1d',  end='2021-06-01',progress=False)
df = yf.download('WMT',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False)
df = df.append(yf.download('AMZN',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('AAPL',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('CVS',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('UNH',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('BRK-A',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('MCK',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('ABC',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('GOOGL',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))
df = df.append(yf.download('XOM',start='2016-06-01', interval='1d',  end='2021-07-13',progress=False))

df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
df["Date"] = df.index
df.head()
print(len(df))

tf = yf.download('GE',start='2020-07-13', interval='1d',  end='2021-07-13',progress=False)
tf.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
tf["Date"] = tf.index
tf.head()
 
#data.plot(figsize=(10,10))

12870


,Close,Adj Close,Date
Date,,,
2020-07-13,6.70,6.672793,2020-07-13
2020-07-14,6.88,6.852062,2020-07-14
2020-07-15,7.14,7.111006,2020-07-15
2020-07-16,7.05,7.021373,2020-07-16
2020-07-17,7.07,7.041291,2020-07-17


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12870 entries, 2016-06-01 to 2021-07-12
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Close      12870 non-null  float64       
 1   Adj Close  12870 non-null  float64       
 2   Date       12870 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 402.2 KB


In [34]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

test_data = tf['Close'].values
test_data = test_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data
close_test = test_data

date_train = df['Date']
date_test = tf['Date']

print(len(close_train))
print(len(close_test))

12870
252


In [35]:
look_back = 4

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=40)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [36]:
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mae')

num_epochs = 30
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)
model.save('stock_model.h5',include_optimizer=True)

Epoch 1/30
322/322 [==============================] - 1s 3ms/step - loss: 13070.8612
Epoch 2/30
322/322 [==============================] - 1s 2ms/step - loss: 1402.3090
Epoch 3/30
322/322 [==============================] - 1s 2ms/step - loss: 518.4107
Epoch 4/30
322/322 [==============================] - 1s 2ms/step - loss: 570.7751
Epoch 5/30
322/322 [==============================] - 1s 2ms/step - loss: 579.1408
Epoch 6/30
322/322 [==============================] - 1s 2ms/step - loss: 576.1836
Epoch 7/30
322/322 [==============================] - 1s 2ms/step - loss: 580.8273
Epoch 8/30
322/322 [==============================] - 1s 2ms/step - loss: 529.0954
Epoch 9/30
322/322 [==============================] - 1s 2ms/step - loss: 627.4241
Epoch 10/30
322/322 [==============================] - 1s 2ms/step - loss: 550.0809
Epoch 11/30
322/322 [==============================] - 1s 2ms/step - loss: 538.2179
Epoch 12/30
322/322 [==============================] - 1s 2ms/step - loss: 553.235

In [37]:
print(model)

In [38]:
#Test
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "GE Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace2, trace3], layout=layout)
fig.show()

In [39]:
# close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = test_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = tf['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)
print(forecast_dates) 

[Timestamp('2021-07-12 00:00:00', freq='D'), Timestamp('2021-07-13 00:00:00', freq='D'), Timestamp('2021-07-14 00:00:00', freq='D'), Timestamp('2021-07-15 00:00:00', freq='D'), Timestamp('2021-07-16 00:00:00', freq='D'), Timestamp('2021-07-17 00:00:00', freq='D'), Timestamp('2021-07-18 00:00:00', freq='D'), Timestamp('2021-07-19 00:00:00', freq='D'), Timestamp('2021-07-20 00:00:00', freq='D'), Timestamp('2021-07-21 00:00:00', freq='D'), Timestamp('2021-07-22 00:00:00', freq='D'), Timestamp('2021-07-23 00:00:00', freq='D'), Timestamp('2021-07-24 00:00:00', freq='D'), Timestamp('2021-07-25 00:00:00', freq='D'), Timestamp('2021-07-26 00:00:00', freq='D'), Timestamp('2021-07-27 00:00:00', freq='D'), Timestamp('2021-07-28 00:00:00', freq='D'), Timestamp('2021-07-29 00:00:00', freq='D'), Timestamp('2021-07-30 00:00:00', freq='D'), Timestamp('2021-07-31 00:00:00', freq='D'), Timestamp('2021-08-01 00:00:00', freq='D'), Timestamp('2021-08-02 00:00:00', freq='D'), Timestamp('2021-08-03 00:00:00'

In [40]:
trace1 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "GE Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
pfig = go.Figure(data=[trace1, trace2], layout=layout)
pfig.show()